In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('mytree').getOrCreate()

In [2]:
data = spark.read.csv('/FileStore/tables/dog_food.csv', inferSchema=True, header=True)

In [3]:
data.show()

+---+---+----+---+-------+
 A| B| C| D|Spoiled|
+---+---+----+---+-------+
 4| 2|12.0| 3| 1.0|
 5| 6|12.0| 7| 1.0|
 6| 2|13.0| 6| 1.0|
 4| 2|12.0| 1| 1.0|
 4| 2|12.0| 3| 1.0|
 10| 3|13.0| 9| 1.0|
 8| 5|14.0| 5| 1.0|
 5| 8|12.0| 8| 1.0|
 6| 5|12.0| 9| 1.0|
 3| 3|12.0| 1| 1.0|
 9| 8|11.0| 3| 1.0|
 1| 10|12.0| 3| 1.0|
 1| 5|13.0| 10| 1.0|
 2| 10|12.0| 6| 1.0|
 1| 10|11.0| 4| 1.0|
 5| 3|12.0| 2| 1.0|
 4| 9|11.0| 8| 1.0|
 5| 1|11.0| 1| 1.0|
 4| 9|12.0| 10| 1.0|
 5| 8|10.0| 9| 1.0|
+---+---+----+---+-------+
only showing top 20 rows

In [4]:
from pyspark.ml.feature import VectorAssembler

In [5]:
assembler = VectorAssembler(inputCols=['A', 'B', 'C', 'D'], 
                            outputCol='features')

In [6]:
output = assembler.transform(data)

In [7]:
output.show()

+---+---+----+---+-------+-------------------+
 A| B| C| D|Spoiled| features|
+---+---+----+---+-------+-------------------+
 4| 2|12.0| 3| 1.0| [4.0,2.0,12.0,3.0]|
 5| 6|12.0| 7| 1.0| [5.0,6.0,12.0,7.0]|
 6| 2|13.0| 6| 1.0| [6.0,2.0,13.0,6.0]|
 4| 2|12.0| 1| 1.0| [4.0,2.0,12.0,1.0]|
 4| 2|12.0| 3| 1.0| [4.0,2.0,12.0,3.0]|
 10| 3|13.0| 9| 1.0|[10.0,3.0,13.0,9.0]|
 8| 5|14.0| 5| 1.0| [8.0,5.0,14.0,5.0]|
 5| 8|12.0| 8| 1.0| [5.0,8.0,12.0,8.0]|
 6| 5|12.0| 9| 1.0| [6.0,5.0,12.0,9.0]|
 3| 3|12.0| 1| 1.0| [3.0,3.0,12.0,1.0]|
 9| 8|11.0| 3| 1.0| [9.0,8.0,11.0,3.0]|
 1| 10|12.0| 3| 1.0|[1.0,10.0,12.0,3.0]|
 1| 5|13.0| 10| 1.0|[1.0,5.0,13.0,10.0]|
 2| 10|12.0| 6| 1.0|[2.0,10.0,12.0,6.0]|
 1| 10|11.0| 4| 1.0|[1.0,10.0,11.0,4.0]|
 5| 3|12.0| 2| 1.0| [5.0,3.0,12.0,2.0]|
 4| 9|11.0| 8| 1.0| [4.0,9.0,11.0,8.0]|
 5| 1|11.0| 1| 1.0| [5.0,1.0,11.0,1.0]|
 4| 9|12.0| 10| 1.0|[4.0,9.0,12.0,10.0]|
 5| 8|10.0| 9| 1.0| [5.0,8.0,10.0,9.0]|
+---+---+----+---+-------+-------------------+
only showing top 20 rows

In [8]:
final_data = output.select('features', 'Spoiled')

In [9]:
final_data.select('Spoiled').describe().show()

+-------+-------------------+
summary| Spoiled|
+-------+-------------------+
 count| 490|
 mean| 0.2857142857142857|
 stddev|0.45221563164613465|
 min| 0.0|
 max| 1.0|
+-------+-------------------+

In [10]:
final_data.groupBy('Spoiled').count().show()

+-------+-----+
Spoiled|count|
+-------+-----+
 0.0| 350|
 1.0| 140|
+-------+-----+

In [11]:
from pyspark.ml.classification import RandomForestClassifier, GBTClassifier, DecisionTreeClassifier

In [12]:
rfc = RandomForestClassifier(numTrees=150, labelCol='Spoiled', featuresCol='features')

In [13]:
rfc_model = rfc.fit(final_data)

In [14]:
# index represents the features. A is 0, B is 1, C is 2, D is 3
rfc_model.featureImportances

Out[24]: SparseVector(4, {0: 0.0188, 1: 0.0179, 2: 0.9411, 3: 0.0223})

In [15]:
# C is the most important feature